In [ ]:
# O objetivo aqui vai ser arrumar os dados para backtest

In [1]:
import pandas as pd
import numpy as np
# https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html

In [ ]:
# Algumas questões sobre markdown
# https://tex.stackexchange.com/questions/130510/write-text-correctly-in-equations #\mathrm and its friends
# https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html
# https://www.ibm.com/support/knowledgecenter/en/SSGNPV_1.1.3/dsx/markd-jupyter.html

The magic formula ranks companies based on two factors: return on capital and earnings yield.

#### Return on Capital
$\textrm{ROC} = \textrm{EBIT}/\textrm{(Net Working Capital + Net Fixed Assets)}$

$\begin{split}
\textrm{Net Fixed Assets} & = \textrm{Total Assets} \\
 & - \textrm{Total Current Assets} \\
 & - \textrm{Total Intangibles & Goodwill}
\end{split}$

#### Earnings yield
$\textrm{Earnings Yield} = \textrm{EBIT} / \textrm{Enterprise Value}$

$\begin{split}
\textrm{Enterprise Value} & = \textrm{Market Value of Equity} \\
 & + \textrm{Net Interest-Bearing Debt}
\end{split}$

In [2]:
def org_columns(df):
    '''
    Organiza os dataframes com fundamentos:
        Renomeia columns
        Seleciona apenas tickers de interesse
    '''
    tickers = df.columns.str.extract('(.*)$')[0]
    df.columns = tickers
    on = tickers[tickers.str.contains('.\D3$')]
    pn = tickers[tickers.str.contains('.\D4$')]
    sto_tick = pd.concat([on, pn], ignore_index=True)
    df = df[sto_tick]
    df = df[df.index.str.contains("Mar|Jun|Sep|Dec")]
    df.columns.name = "ticker"
    return df

In [3]:
# EBIT
ebit_ati = pd.read_excel('../../data/economatica_1/ebit_mensal__ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ebit_can = pd.read_excel('../../data/economatica_1/ebit_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ebit_can.drop(["Jan-86", "Feb-86"], inplace=True)

ebit = pd.concat([ebit_ati,ebit_can],axis=1,sort=False)
ebit = org_columns(ebit)

In [4]:
# Net Working Capital
nwc_ati = pd.read_excel('../../data/economatica_1/working_capital_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nwc_can = pd.read_excel('../../data/economatica_1/working_capital_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nwc_can.drop(["Jan-86", "Feb-86"], inplace=True)

nwc = pd.concat([ebit_ati,ebit_can],axis=1,sort=False)
nwc = org_columns(nwc)

In [5]:
# Total Assets
ta_ati = pd.read_excel('../../data/economatica_1/total_assets_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ta_can = pd.read_excel('../../data/economatica_1/total_assets_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ta_can.drop(["Jan-86", "Feb-86"], inplace=True)

ta = pd.concat([ta_ati,ta_can],axis=1,sort=False)
ta = org_columns(ta)

In [6]:
# Ativo Circulante
ac_ati = pd.read_excel('../../data/economatica_1/ativo_circulante_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ac_can = pd.read_excel('../../data/economatica_1/ativo_circulante_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
ac_can.drop(["Jan-86", "Feb-86"], inplace=True)

ac = pd.concat([ac_ati,ac_can],axis=1,sort=False)
ac = org_columns(ac)

In [7]:
# Passivo Circulante
pc_ati = pd.read_excel('../../data/economatica_1/passivo_circulante_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
pc_can = pd.read_excel('../../data/economatica_1/passivo_circulante_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
pc_can.drop(["Jan-86", "Feb-86"], inplace=True)

pc = pd.concat([pc_ati,pc_can],axis=1,sort=False)
pc = org_columns(pc)

In [8]:
# Total Current Assets
tca = ac - pc

In [9]:
# Goodwill
gw_ati = pd.read_excel('../../data/economatica_1/goodwill_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
gw_can = pd.read_excel('../../data/economatica_1/goodwill_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
# pc_can.drop(["Jan-86", "Feb-86"], inplace=True)

gw = pd.concat([gw_ati,gw_can],axis=1,sort=False)
gw = org_columns(gw)

In [55]:
x = gw.loc["Mar-19"]
y = ~x.isnull()
y.sum()

2

In [56]:
# Net Fixed Assets
nfa = ta - tca - gw

In [12]:
# Market Value of Equity
mve_ati = pd.read_excel('../../data/economatica_1/valor_mercado_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
mve_can = pd.read_excel('../../data/economatica_1/valor_mercado_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
mve_can.drop(["Jan-86", "Feb-86"], inplace=True)

mve = pd.concat([mve_ati,mve_can],axis=1,sort=False)
mve = org_columns(mve)

In [13]:
# Net Interest-Bearing Debt
nibd_ati = pd.read_excel('../../data/economatica_1/net_debt_mensal_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nibd_can = pd.read_excel('../../data/economatica_1/net_debt_mensal_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
nibd_can.drop(["Jan-86", "Feb-86"], inplace=True)

nibd = pd.concat([nibd_ati,nibd_can],axis=1,sort=False)
nibd = org_columns(nibd)

In [14]:
# Enterprise Level
ev = mve + nibd

In [11]:
# Return on Capital (ROC)
roc = ebit / (nwc + nfa)

In [15]:
# Earnings Yield
ey = ebit/ev

In [38]:
# ey.loc["Mar-18"]
ey_ = ey.loc["Sep-19"]
roc_ = roc.loc["Mar-19"]

In [36]:
roc_.sort_values(ascending=False)

ticker
AALR3   NaN
ABCB3   NaN
ABCB4   NaN
ABEV3   NaN
ABYA3   NaN
ACES3   NaN
ACES4   NaN
ACGU3   NaN
ADHM3   NaN
AEDU3   NaN
AEDU4   NaN
AELP3   NaN
AESL3   NaN
AESL4   NaN
AETA3   NaN
AFLT3   NaN
AFLU3   NaN
AGEI3   NaN
AGIN3   NaN
AGRA4   NaN
AGRO3   NaN
AHEB3   NaN
ALBA3   NaN
ALLL3   NaN
ALLL4   NaN
ALPA3   NaN
ALPA4   NaN
ALSC3   NaN
ALSO3   NaN
ALTR3   NaN
         ..
VLID3   NaN
VPSC3   NaN
VPSC4   NaN
VPTA3   NaN
VPTA4   NaN
VRLM4   NaN
VSMA3   NaN
VTEC4   NaN
VULC3   NaN
VULC4   NaN
VVAR3   NaN
VVAR4   NaN
VVAX3   NaN
VVAX4   NaN
WEGE3   NaN
WEGE4   NaN
WET4    NaN
WHMT3   NaN
WHRL3   NaN
WHRL4   NaN
WISA3   NaN
WISA4   NaN
WIZS3   NaN
WLMM3   NaN
WLMM4   NaN
WMBY3   NaN
WMBY4   NaN
WWOW3   NaN
YDUQ3   NaN
ZIVI4   NaN
Name: Sep-19, Length: 999, dtype: float64

In [45]:
# criar df com dictionary e um index e usar expressões lógicas
df1 = pd.DataFrame({'A': [12, 44, 17, 1],
                    'B': [4, 8, 100, -3],
                    'C': [47, 68, 52, 1],
                    'D': [0, 0, 0, 0]},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': [1, 2, 3, 4],
                    'B': [-10, 4, 5, 13],
                    'C': [12, 23, 10, 100],
                    'D': [1, 1, 1, 1]},
                    index=[0, 1, 2, 3])
df3 = pd.DataFrame({'A': [1, 2, 3, 4],
                    'B': [-10, np.nan, 5, 13],
                    'C': [np.nan, 23, 10, 100],
                    'D': [1, 1, 1, 1]},
                    index=[0, 1, 2, 3])